In [ ]:
from app.service.text_service import TextAnalysisService
from app.service.image_service import ImageAnalysisService
from app.service.file_service import FileAnalysisService
from app.service.url_services import UrlAnalysisService
from app.service.label_service import LabelApplicationService, LabelManagementService

from bson import ObjectId

text_analysis_service = TextAnalysisService()
url_analysis_service = UrlAnalysisService()
file_analysis_service = FileAnalysisService()
image_analysis_service = ImageAnalysisService()

await text_analysis_service.process_text()
await url_analysis_service.process_urls()
await file_analysis_service.process_files()
await image_analysis_service.process_images()

In [1]:
from pymongo import MongoClient
from app.infrastructure.external.cloudflare_ai_service import CloudflareAIService
from app.infrastructure.daos.text_daos import TextDAO
from app.infrastructure.daos.image_daos import ImageDAO
from app.infrastructure.daos.user_daos import UserDAO

In [2]:
from app.service.user_service import UserManagementService
user_management_service = UserManagementService()
await user_management_service.get_users_by_line_group_id("Ced6be955b978ec9e38693641bca5eae2")

[ObjectId('67e3c391ee7d33ce57110243')]

In [9]:
llm_service = CloudflareAIService()

dao = TextDAO()
dao = ImageDAO()

query_text = "GPU"

result = await dao.full_text_search(query_text)

# query_vector = await llm_service.get_embedding(query_text)
# result = await dao.vector_search(query_vector)

In [ ]:
result